## 2022-10-2122 SN 2022vqz

Analysis by Brian

### Non-Standard Calibration

We are using a slightly non-standard but very clean calibration procedure which requires
the lights and the darks to have the same exposure duration (typically of order 30s),
and the 

In [1]:
import os
import numpy as np
from astropy import units as u
from astropy.nddata import CCDData
from ccdproc import ImageFileCollection, Combiner, combine, subtract_dark, flat_correct

In [2]:
observation_date = '2022-10-2425'
light_exposure = 30 * u.second
dark_exposure = 30 * u.second
flat_exposure = 1 * u.second
bias_exposure = 1 * u.second

In [3]:
data_directory = os.path.join(os.path.expanduser('~'), 'Transients', observation_date)

In [4]:
light_directory = os.path.join(data_directory, 'light')
dark_directory = os.path.join(data_directory, 'dark')
flat_directory = os.path.join(data_directory, 'flat')
bias_directory = os.path.join(data_directory, 'bias')

In [5]:
light_files = ImageFileCollection(light_directory).files_filtered(include_path='True')
dark_files = ImageFileCollection(dark_directory).files_filtered(include_path='True')
flat_files = ImageFileCollection(flat_directory).files_filtered(include_path='True')
bias_files = ImageFileCollection(bias_directory).files_filtered(include_path='True')

In [6]:
lights = [CCDData.read(file, unit=u.adu) for file in light_files]
darks = [CCDData.read(file, unit=u.adu) for file in dark_files]
flats = [CCDData.read(file, unit=u.adu) for file in flat_files]
biases = [CCDData.read(file, unit=u.adu) for file in bias_files]

Set OBSGEO-Y to -4483197.997 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851226.198 from OBSGEO-[LBH]'. [astropy.wcs.wcs]
Set OBSGEO-Y to -4483197.997 from OBSGEO-[LBH].
Set OBSGEO-Z to  3851226.198 from OBSGEO-[LBH]'.


In [7]:
master_dark = combine(darks)
master_flat = combine(flats)
master_bias = combine(biases)

In [8]:
lights_subtracted = [subtract_dark(light, master_dark, data_exposure=light_exposure, dark_exposure=dark_exposure, scale=False) for light in lights]
master_flat_subtracted = subtract_dark(master_flat, master_bias, data_exposure=flat_exposure, dark_exposure=bias_exposure, scale=False)

In [9]:
lights_calibrated = [flat_correct(light_subtracted, master_flat_subtracted) for light_subtracted in lights_subtracted]

/Applications/Anaconda/anaconda3/envs/transient/lib/python3.9/site-packages/astropy/units/quantity.py:611: RuntimeWarning: invalid value encountered in true_divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
/Applications/Anaconda/anaconda3/envs/transient/lib/python3.9/site-packages/astropy/nddata/nduncertainty.py:684: RuntimeWarning: invalid value encountered in true_divide
  return from_variance((left + right + correlation_sign * corr) /


In [13]:
first_light = lights[0]

In [14]:
first_light

CCDData([[532, 533, 524, ..., 522, 530, 518],
         [527, 523, 525, ..., 536, 523, 532],
         [534, 527, 533, ..., 524, 518, 526],
         ...,
         [524, 529, 529, ..., 525, 526, 518],
         [531, 523, 527, ..., 516, 516, 528],
         [  0,   0,   0, ...,   0,   0,   0]], unit='adu')

In [15]:
first_light_calibrated = lights_calibrated[0]

In [16]:
first_light_calibrated

CCDData([[21.9794809 , 25.42397942, 12.79140773, ..., 12.21091556,
          25.5986195 ,  7.82844981],
         [14.4002259 , 13.88914881, 13.92323643, ..., 34.79273486,
          12.23754335, 28.34568314],
         [24.20065907, 14.6629941 , 21.55910118, ..., 14.5033401 ,
           8.41162298, 17.0001092 ],
         ...,
         [13.75749183, 18.2692002 , 19.67782442, ..., 19.83836336,
          20.2521573 , 10.05484021],
         [20.1395336 , 13.8793963 , 15.05366152, ...,  4.2865294 ,
           3.82474731, 24.85372615],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan]], unit='adu')